# Primer uporabe orodja AutoGen


## Uvoz potrebnih paketov


In [ ]:
import os
import json

import requests
from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_core.tools import FunctionTool
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from typing import Any, Callable, Set, Dict, List, Optional

## Ustvarjanje odjemalca

V tem primeru bomo uporabili [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) za dostop do LLM.

`model` je definiran kot `gpt-4o-mini`. Poskusite zamenjati model z drugim modelom, ki je na voljo na tržnici GitHub Models, da vidite različne rezultate.

Za hiter preizkus bomo izvedli preprost poziv - `Kaj je glavno mesto Francije`.


In [ ]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## Definiranje funkcij

V tem primeru bomo agentu omogočili dostop do orodja, ki je funkcija s seznamom razpoložljivih počitniških destinacij in njihovo razpoložljivostjo.

Lahko si predstavljate, da gre za scenarij, kjer bi imel potovalni agent na primer dostop do potovalne baze podatkov.

Ko pregledujete ta primer, lahko poskusite definirati nove funkcije in orodja, ki jih lahko agent uporabi.


In [ ]:
from typing import Dict, List, Optional


def vacation_destinations(city: str) -> tuple[str, str]:
    """
    Checks if a specific vacation destination is available
    
    Args:
        city (str): Name of the city to check
        
    Returns:
        tuple: Contains city name and availability status ('Available' or 'Unavailable')
    """
    destinations = {
        "Barcelona": "Available",
        "Tokyo": "Unavailable",
        "Cape Town": "Available",
        "Vancouver": "Available",
        "Dubai": "Unavailable",
    }

    if city in destinations:
        return city, destinations[city]
    else:
        return city, "City not found"

# Example usage:
# city, status = vacation_destinations("Barcelona")
# print(f"How about visiting {city}? It's currently {status} there!")

## Definiranje orodja FunctionTool  
Da lahko agent uporablja `vacation_destinations` kot `FunctionTool`, ga moramo definirati kot takega.  

Prav tako bomo zagotovili opis orodja, ki bo agentu pomagal prepoznati, za kaj se to orodje uporablja v povezavi z nalogo, ki jo je uporabnik zahteval.  


In [ ]:
get_vacations = FunctionTool(
    vacation_destinations, description="Search for vacation destinations and if they are available or not."
)

## Definiranje agenta

Zdaj lahko ustvarimo agenta v spodnji kodi. Določimo `system_message`, da agentu podamo navodila, kako pomagati uporabnikom pri iskanju počitniških destinacij.

Prav tako nastavimo parameter `reflect_on_tool_use` na true. To omogoča, da LLM uporabi odgovor orodja in ga pošlje v naravnem jeziku.

Parameter lahko nastavite na false, da vidite razliko.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[get_vacations],
    system_message="You are a travel agent that helps users find vacation destinations.",
    reflect_on_tool_use=True,
)

## Zagon agenta

Zdaj lahko zaženemo agenta z začetnim sporočilom uporabnika, ki sprašuje o potovanju v Tokio.

Lahko spremenite to destinacijo mesta, da preverite, kako agent odgovarja glede razpoložljivosti mesta.


In [ ]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="I would like to take a trip to Tokyo", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()


---

**Omejitev odgovornosti**:  
Ta dokument je bil preveden z uporabo storitve za prevajanje z umetno inteligenco [Co-op Translator](https://github.com/Azure/co-op-translator). Čeprav si prizadevamo za natančnost, vas prosimo, da upoštevate, da lahko avtomatizirani prevodi vsebujejo napake ali netočnosti. Izvirni dokument v njegovem maternem jeziku je treba obravnavati kot avtoritativni vir. Za ključne informacije priporočamo profesionalni človeški prevod. Ne prevzemamo odgovornosti za morebitne nesporazume ali napačne razlage, ki bi nastale zaradi uporabe tega prevoda.
